In [183]:
"""

There is a correspondence between finite (T0) topological spaces and finite posets.
The goal is to represent finite top. spaces and also posets in python,
and to be able to convert between them. 

This correspondence is functorial, meaning that continuous functions between topological spaces
correspond to order-preserving functions between posets.

The correspondence is given via the 'specialization order' , which is defined as follows:
x <= y if and only if x is in the closure of the singleton {y}.
Conversely: given a poset, a topology is defined by the basic open sets
 U_x := {y | x <= y} 

Some functions I want to implement:
- Homology groups of a topological space
- Barycentric subdivision 
- Cartesian product of topological spaces
- Mapping space Hom(X,Y) of continuous functions between topological spaces
- Simplicial complexes
- Cone
- Suspension

An long-term goal is to use this for topological quantum error correction codes.
i.e. for each point in the space, there is a qubit, and the topology of the space
determines the stabilizer generators. 

"""
import itertools
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import simplicial as simp
import galois as gal

class Top:
    ''' 

    '''
    
    def __init__(self, data):

        if type(data) == nx.DiGraph:
            # basis is dictionary of the basic open sets { key = x : value = {y | x <= y} }
            basis = {}
            for node in data.nodes:
                basis[node] = set(nx.descendants(data,node)).union({node})
            self.basis = tuple(basis)
            self.points = set(data.nodes)

            

        elif type(data) == dict:
            self.basis = data
            self.points = set(data.keys())

    def cl(self,x):
        ''' 
        Returns the closure of a point x in the topological space: cl{x}
        '''
        s = set()
        for point in self.points:
            if x not in self.basis[point]:
                s = s.union(set(self.basis[point]))
        return self.points.difference(s)
    
    def isleq(self,x,y):
        ''' 
        Returns True if x <= y, False otherwise
        with respect to the specialization order
        '''
        return x in self.cl(y)
    
    def ischain(self, subset):
        ''' 
        Given a subset of the topological space, returns True iff it is a chain
        (meaning each element is comparable to every other)
        '''
        chain = True
        for point in subset:
            for other in subset:
                chain = chain and (self.isleq(point,other) or self.isleq(other,point))
        return chain
    
    def all_chains(self):
        
        """ 
        
        Returns an array of the set of chains (x_0 <= x_1 <= ... <= x_n ) 
        w.r.t the specialization order of the topological space.

        The chains are returned as a tuple of tuples, where the chains are ordered as follows:
            - The chains are listed in increasing order of length
            - Each individual chain is ordered according to the specialization order

        for example, if the topological space is {0:{0,1,2},1:{1,2},2:{2}}, then the output is:
        [[0], [1], [2], [0, 1], [0, 2], [1, 2], [0, 1, 2]]
        
        """
        chains = []
        for n in range(1,len(self.points)+1):
            for chain in itertools.combinations(self.points,n):
                if self.ischain(chain):
                    chain_as_list = list(chain)

                    # put the chain in order according to the specialization order
                    for i in range(len(chain_as_list)-1):
                        min = i
                        for j in range(i+1,len(chain_as_list)):
                            if self.isleq(chain_as_list[j],chain_as_list[min]):
                                min = j
                        
                        chain_as_list[i],chain_as_list[min] = chain_as_list[min],chain_as_list[i]
                    
                        

                    chains.append(tuple(chain_as_list))
        return tuple(chains)

def Top_to_graph(T : Top) -> nx.DiGraph:
    ''' 
    Given a topological space T, returns the poset corresponding to the specialization order.
    Here, the poset is represented as a directed graph via the networkx library.
    x<=y in the poset iff there exists an edge connecting between x and y in the graph
    
    '''
    G = nx.DiGraph()
    for x in T.points:
        for y in T.points:
            if T.isleq(x,y):
                G.add_edge(x,y)
    return G
                    
def Top_to_reduced_graph(T : Top) -> nx.DiGraph:
    ''' 
    Given a topological space T, returns the poset corresponding to the specialization order.
    Here, the poset is represented as a directed graph via the networkx library.
    x<=y in the poset iff there exists a directed path from x to y in the graph
    (including the length 0 path x->x).

    Currently, this will return an error if the topological space is not T_0,
    since in that case the specialization order is not a partial order, but a pre-order.
    Ideally, I would like to implement a function that returns the T_0 quotient of a topological space.
    '''
    G = nx.DiGraph()
    for node in T.points:
        G.add_node(node)
    for x in T.points:
        for y in T.points:
            if T.isleq(x,y) and x != y:
                G.add_edge(x,y)
    G = nx.transitive_reduction(G)
    
    return G






# G = nx.DiGraph()
# G.add_nodes_from([1,2,3,4])
# G.add_edges_from([(1,3),(1,4),(2,3),(2,4)])




T = Top({0:(0,1,2),1:(1,2),2:(2,)})
print( T.all_chains() )

# G = Top_to_graph(T)
# nx.draw(G,label=True,with_labels=True)
# plt.show

((0,), (1,), (2,), (0, 1), (0, 2), (1, 2), (0, 1, 2))


In [213]:

class SimpComp:

    ''' 

    Encode a simplicial complex as a list of simplices, where each simplex is a list of vertices.
    '''

    def __init__(self, data):
        self.simplices = data
        self.dim = max([len(simplex) for simplex in data]) - 1

    def faces(self, n = None):
        ''' 
        If n is None, returns a tuple of all faces indexed by dimension, i.e.
        ((0-faces),(1-faces),...,(n-faces))

        If n is an integer, returns the set of n-dimensional faces of the simplicial complex
        (n-faces)
        '''
        if type(n) == int:
            return tuple([simplex for simplex in self.simplices if len(simplex) == n+1])
        
        elif n == None:
            faces = tuple([self.faces(i) for i in range(self.dim+1)])
            return faces
        else :
            raise ValueError("n must be an integer or None")
    def points(self):
        return tuple([x[0] for x in self.faces(0)])
    
    def skeleton(self,n:int):
        ''' 
        Returns the n-skeleton of the simplicial complex
        '''
        return SimpComp([simplex for simplex in self.simplices if len(simplex) <= n+1])
    
def is_sublist(l1,l2):
    ''' 
    Returns True if l1 is a sublist of l2, False otherwise
    '''
    return all([x in l2 for x in l1])

def is_chain_of_lists(L):
    ''' 
    Returns True if L is a chain of lists, False otherwise
    '''
    is_chain = True
    for l in L:
        for m in L:
            is_chain = is_chain and (is_sublist(l,m) or is_sublist(m,l))
    return is_chain
    
def bary_subd(S:SimpComp):
    ''' 
    Given a simplicial complex S, returns the barycentric subdivision of S
    '''
    new_vertices = S.simplices
    new_simplices = []
    for n in range(1,S.dim+2):
        for tuples in itertools.combinations(new_vertices,n):
            if is_chain_of_lists(tuples):
                new_simplices.append(tuples)
    return SimpComp(tuple(new_simplices))    


def Top_to_simp(T:Top) -> SimpComp:
    ''' 
    Given a topological space T, returns the simplicial complex corresponding to the specialization order.
    '''
    return SimpComp(T.all_chains())

def Simp_to_top(S:SimpComp) -> Top:
    ''' 
    Given a simplicial complex S, returns the topological space corresponding to the specialization order.
    '''
    # the points of X will be the simplices of B = bary_subd(S)
    B = bary_subd(S)
    basis = {}
    # the basic open of sigma in B is {tau | sigma is a face of tau}
    for simplex in S.simplices:
        is_face_of = []
        for simplex2 in S.simplices:
            if is_sublist(simplex,simplex2):
                is_face_of.append(simplex2)
        # put the open set in order according to inclusion
        for i in range(len(is_face_of)-1):
            min = i
            for j in range(i+1,len(is_face_of)):
                if is_sublist(is_face_of[j],is_face_of[min]):
                    min = j
            
            is_face_of[i],is_face_of[min] = is_face_of[min],is_face_of[i]
        basis[simplex] = tuple(is_face_of)
    return Top(basis)

def boundary(simplex):
    ''' 
    Given a simplex, returns the boundary of the simplex as a tuple. There is an implied (-1)^n, i.e.
    boundary((a,b,c)) = ((b,c) , (a,c) , (a,b)) =: (b,c) - (a,c) + (a,b) 
    '''
    return tuple([simplex[:i] + simplex[i+1:] for i in range(len(simplex))])

def boundary_map(S: SimpComp , n : int):
    ''' 
    Returns the nth boundary map as a matrix
    '''
    d_n = np.zeros((len(S.faces(n-1)),len(S.faces(n))))
    for i,face in enumerate(S.faces(n)):
        for j,face2 in enumerate(S.faces(n-1)):
            if face2 in boundary(face):
                d_n[j,i] = (-1) ** (boundary(face).index(face2))
    return d_n

def Betti(S : SimpComp, coeff = 'Z') -> tuple[int,... ]:
    ''' 
    Given a simplicial complex S, returns the Betti numbers of S with specifice coefficients.
    The betti number b_n is the nth rank of the simplicial homology group H_n(S;coeff)

    By default, the coefficients are taken to be integers. 
    For a finite field GF(p**n), coeff = p**n
    '''
    betti = [0 for i in range(S.dim +1)]
    
    dims = [len(S.faces(i)) for i in range(S.dim+1)]
    for n in range(S.dim+1):
        if coeff == 'Z':
            if n == 0:
                betti[n] = dims[n] - np.linalg.matrix_rank(boundary_map(S,n+1))
            elif n == S.dim:
                betti[n] = dims[n] - np.linalg.matrix_rank(boundary_map(S,n))
            else:
                betti[n] = dims[n] - np.linalg.matrix_rank(boundary_map(S,n))-np.linalg.matrix_rank(boundary_map(S,n+1))

        elif coeff == p:
            GF = gal.GF(p)
            betti[n] = dims[n] - np.linalg.matrix_rank(GF(boundary_map(S,n)))-np.linalg.matrix_rank(GF(boundary_map(S,n+1)))


    return tuple(betti)
    

In [ ]:
"""
A class for continuous functions between topological spaces
"""
class Func: 

    def __init__(self, f, X : Top, Y : Top):
        self.graph = f
        self.domain = X
        self.codomain = Y
    """
    f is a dictionary of the form {x : f(x)}
    """
    
    def iscont(self):
        """
        Returns True if the function is continuous

        There are a couple things to check:
        1. f is actually a function, i.e.
          a. if x in X, then f.graph[x] is in Y.points
          b. if f.graph[x_1]=f.graph[x_2], then x_1 = x_2
        2. f is continuous with respect to the topologies of X and Y
          a. if x_1 <= x_2 in X, then f.graph[x_1] <= f.graph[x_2] in Y
        """
        if self.graph.keys() != self.domain.points:
            print ("That's not a function!" + " The domain is wrong!")
            return False 
        for x in self.domain.points:
            if self.graph[x] not in self.codomain.points:
                print ("That's not a function!" + " The codomain is wrong!")
                return False
        
        for x in self.domain.points:
            for y in self.domain.points:
                if self.domain.isleq(x,y) and not self.codomain.isleq(self.graph[x],self.graph[y]):
                    return False
        return True

# test a few examples
# S = Serpinski space
S = Top({0:{0,1},1:{1}})
# id should be continuous
id = Func({0:0,1:1},S,S)
print(id.iscont())
# example of something that isn't continuous
f = Func({0:1,1:0},S,S)
print(f.iscont())
# examples of a non-functions
g= Func({0:1,1:2},S,S)
h= Func({0:0},S,S)
print(g.iscont())
print(h.iscont())

True
False
That's not a function! The codomain is wrong!
False
That's not a function! The domain is wrong!
False


In [224]:
# S = SimpComp([[0],[1],[2],[0,1],[0,2],[1,2],[0,1,2]])
S = SimpComp([(0,),(1,),(2,),(3,),(0,1),(0,2),(0,3),(1,2),(1,3),(2,3),(0,1,2),(0,1,3),(1,2,3),(0,2,3)])
T = Simp_to_Top(S)
B = Top_to_simp(T)
# G = Top_to_reduced_graph(T)
# nx.draw(G,label=True,with_labels=True)

T2 = Top({0:(0,1,3),2:(1,2,3),1:(1,),3:(3,)})
Betti(Top_to_simp(T2))

(1, 1)

In [225]:
"""
A class for continuous functions between topological spaces
"""
class Func: 

    def __init__(self, f, X : Top, Y : Top):
        self.graph = f
        self.domain = X
        self.codomain = Y
    """
    f is a dictionary of the form {x : f(x)}
    """
    
    def iscont(self):
        """
        Returns True if the function is continuous

        There are a couple things to check:
        1. f is actually a function, i.e.
          a. if x in X, then f.graph[x] is in Y.points
          b. if f.graph[x_1]=f.graph[x_2], then x_1 = x_2
        2. f is continuous with respect to the topologies of X and Y
          a. if x_1 <= x_2 in X, then f.graph[x_1] <= f.graph[x_2] in Y
        """
        if self.graph.keys() != self.domain.points:
            print ("That's not a function!" + " The domain is wrong!")
            return False 
        for x in self.domain.points:
            if self.graph[x] not in self.codomain.points:
                print ("That's not a function!" + " The codomain is wrong!  ... Bakaaaa")
                return False
        
        for x in self.domain.points:
            for y in self.domain.points:
                if self.domain.isleq(x,y) and not self.codomain.isleq(self.graph[x],self.graph[y]):
                    return False
        return True

# test a few examples
# S = Serpinski space
S = Top({0:{0,1},1:{1}})
# id should be continuous
id = Func({0:0,1:1},S,S)
print(id.iscont())
# example of something that isn't continuous
f = Func({0:1,1:0},S,S)
print(f.iscont())
# examples of a non-functions
g= Func({0:1,1:2},S,S)
h= Func({0:0},S,S)
print(g.iscont())
print(h.iscont())

True
False
That's not a function! The codomain is wrong!  ... Bakaaaa
False
That's not a function! The domain is wrong!
False
